In [ ]:
#!pip install geopandas pandas shapely


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import os
caminho = 'C:\\Users\\Edmilson\\Gama_Workspace2\\ABMS-WP'

# 1. Load the census tract shapefile
setores = gpd.read_file(os.path.join(caminho,'includes\\maps\\itapua\\Mapa_Itapua.shp'))

# 2. Select only the CD_SETOR column and the geometry
setores = setores[['CD_SETOR', 'geometry']]

# 3. Displaying the polygon and its respective sectors
setores.head()

,CD_SETOR,geometry
0,292740805090020,"POLYGON ((-38.36855 -12.94796, -38.36859 -12.9..."
1,292740805090021,"POLYGON ((-38.36738 -12.94851, -38.36739 -12.9..."
2,292740805090022,"POLYGON ((-38.36442 -12.94586, -38.36457 -12.9..."
3,292740805090023,"POLYGON ((-38.36433 -12.94755, -38.36433 -12.9..."
4,292740805090024,"POLYGON ((-38.36165 -12.94856, -38.36168 -12.9..."


In [ ]:
# 4. Upload the CSV file with the supply points
pontos_df = pd.read_csv(os.path.join(caminho,'includes\\Tabela_consumidores_Itapua_convertida.csv'), sep=';')

# 5. Display supply points and their coordinates
pontos_df.head()

,SK_MATRICULA,NM_SUBCATEGORIA,NN_MORADORES,ST_PISCINA,LAT_GEO,LONG_GEO,X,Y
0,124584355,NORMAL,5,0,-12.950571,-38.373685,567933.093052,8.568247e+06
1,152961499,NORMAL,1,0,-12.943031,-38.366573,568706.588075,8.569079e+06
2,125305257,INTERMEDIARIA,8,0,-12.944554,-38.363644,569023.883741,8.568910e+06
3,123863460,NORMAL,3,0,-12.940758,-38.369058,568437.655084,8.569331e+06
4,126026156,NORMAL,2,0,-12.936883,-38.358508,569583.120489,8.569757e+06


In [ ]:

# 6. Convert the Points DataFrame to a GeoDataFrame
geometry = [Point(xy) for xy in zip(pontos_df['LONG_GEO'], pontos_df['LAT_GEO'])]
pontos_gdf = gpd.GeoDataFrame(pontos_df, geometry=geometry, crs="EPSG:4326")  # EPSG:4326 é o código para WGS84


#7. Ensure both GeoDataFrames are in the same coordinate system
pontos_gdf = pontos_gdf.to_crs(setores.crs)

# 8. Perform the spatial join
# This will add the census sector information (geometry and CD_SETOR) to each supply point
resultado = gpd.sjoin(pontos_gdf, setores, how="left", predicate="within")

#9. Delete unnecessary columns
resultado.drop(columns=['geometry','index_right'],inplace=True)

# 10. Delete lines that do not belong to the Itapuã census sectors
resultado.dropna(subset=['CD_SETOR'], inplace=True)

# 11. Save the result to a new CSV (optional)
resultado.to_csv(os.path.join(caminho,'includes\\Tabela_consumidores_Itapua_com_setor.csv'), index=False)


# 12. View the first few lines of the result
print(resultado.head())

   SK_MATRICULA NM_SUBCATEGORIA  NN_MORADORES  ST_PISCINA    LAT_GEO  \
0     124584355          NORMAL             5           0 -12.950571   
1     152961499          NORMAL             1           0 -12.943031   
2     125305257   INTERMEDIARIA             8           0 -12.944554   
3     123863460          NORMAL             3           0 -12.940758   
4     126026156          NORMAL             2           0 -12.936883   

    LONG_GEO              X             Y         CD_SETOR  
0 -38.373685  567933.093052  8.568247e+06  292740805090305  
1 -38.366573  568706.588075  8.569079e+06  292740805090291  
2 -38.363644  569023.883741  8.568910e+06  292740805090067  
3 -38.369058  568437.655084  8.569331e+06  292740805090275  
4 -38.358508  569583.120489  8.569757e+06  292740805090106  


In [8]:
resultado.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 15297 entries, 0 to 15331
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   SK_MATRICULA     15297 non-null  int64  
 1   NM_SUBCATEGORIA  15297 non-null  object 
 2   NN_MORADORES     15297 non-null  int64  
 3   ST_PISCINA       15297 non-null  int64  
 4   LAT_GEO          15297 non-null  float64
 5   LONG_GEO         15297 non-null  float64
 6   X                15297 non-null  float64
 7   Y                15297 non-null  float64
 8   CD_SETOR         15297 non-null  object 
dtypes: float64(4), int64(3), object(2)
memory usage: 1.2+ MB
